In [1]:
# import common packages
import paths
import numpy as np

import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

# lib from QISChem
from qiskit_acqua_chemistry import FermionicOperator

# lib from optimizer and algorithm
from qiskit_acqua.operator import Operator
from qiskit_acqua import (get_algorithm_instance, get_optimizer_instance, get_variational_form_instance)

# lib for driver
from qiskit_acqua_chemistry.drivers import ConfigurationManager
from collections import OrderedDict

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
cfg_mgr = ConfigurationManager()
pyscf_cfg = OrderedDict([('atom', 'H .0 .0 .0; H .0 .0 0.735'), ('unit', 'Angstrom'), ('charge', 0), ('spin', 0), ('basis', 'sto3g')])
section = {}
section['properties'] = pyscf_cfg
driver = cfg_mgr.get_driver_instance('PYSCF')
molecule = driver.run(section)

In [3]:
# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule._one_body_integrals, h2=molecule._two_body_integrals)
qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.convert('paulis','matrix')
qubitOp.chop(10**-10)

In [4]:
# if you do not install any driver and want like to start with random Hamiltonian
# SIZE=4
# matrix = np.random.random((SIZE,SIZE))
# qubitOp = Operator(matrix=matrix)

In [ ]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = get_algorithm_instance('ExactEigensolver')
exact_eigensolver.init_args(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['eigvals'][0].real))

The exact ground state energy is: -1.8572750302023826


In [ ]:
# %timeit
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = get_optimizer_instance('L_BFGS_B')
lbfgs.set_options(maxfun=1000, factr=10, iprint=10)
# spsa = get_optimizer_instance('SPSA')
# lbfgs.print_options()

# setup variation form generator (generate trial circuits for VQE)
var_form = get_variational_form_instance('RYRZ')
var_form.init_args(qubitOp.num_qubits, 5, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variation form, and optimzer
vqe_algorithm = get_algorithm_instance('VQE')
vqe_algorithm.setup_quantum_backend(backend='local_statevector_simulator', skip_translation=True)
vqe_algorithm.init_args(qubitOp, 'matrix', var_form, lbfgs)
results = vqe_algorithm.run()
print("Minimum value: {}".format(results['eigvals'][0].real))
print("Parameters: {}".format(results['opt_params']))